# Parameter definition
Choose all the settings for the database creation.

In [13]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_dataset_chokepoint3' #'colab_main_dataset'
input_dir = os.path.join(home, 'lr_video_project', 'image_datasets')
output_dir = os.path.join(home, 'lr_video_project', 'sql_database')
database_names=['lfw'] # 'lfw', 'scface', 'forenface', 'enfsi', 'enfsi2015' 'xqlfw', 'utkface','chokepoint'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab

## SERFIQ model

Create SERFIQ model.

In [14]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

[16:37:24] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[16:37:24] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[16:37:25] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:37:25] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[16:37:25] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:37:25] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[16:37:25] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:37:25] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[16:37:25] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:37:25] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!


# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [15]:
from sql_face.sqldb import SQLDataBase

In [16]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [17]:
database.__dict__

{'db_name': 'test_video_dataset_chokepoint3',
 'input_dir': '/home/jmacarulla/lr_video_project/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/jmacarulla/lr_video_project/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7ff240cc6350>,
 'databases': [<sql_face.databases.LFW at 0x7ff240d7a410>],
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [18]:
database.create_tables(serfiq=serfiq)

Creating image record from LFW: 0it [00:00, ?it/s]
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [00:00<?, ?it/s][ WARN:0@1051.987] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('lfw/Lleyton_Hewitt/Lleyton_Hewitt_0031.jpg'): can't open/read file: check file path/integrity
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [1:07:28<?, ?it/s]


TypeError: object of type 'NoneType' has no len()

In [ ]:
database.update_tables(attributes_to_update, serfiq = serfiq)